<a href="https://colab.research.google.com/github/mputs/Colab/blob/master/Install_libgpuarray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making PyMC3 use the GPU on google Colab
For PyMC3, i needed theano to be able to use the GPU. However, libgpuarray is not installed. You can see this by running the following code. It will return the text: "Used the **cpu**":

In [0]:
import os
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

For installing libgpuarray, i followed the instructions on http://deeplearning.net/software/libgpuarray/installation.html. The next cells download the repository, build and install libgpuarray, and finally set it up in python.

## clone the git repository
first we clone the git repository:

In [0]:
!git clone https://github.com/Theano/libgpuarray.git

##build libgpuarray
we build libgpuarray using cmake:

In [0]:
!cd libgpuarray; mkdir Build; cd Build; cmake .. -DCMAKE_BUILD_TYPE=Release; make; make install

## build and install libgpuarray
as last stap, we build and install libgpuarray:

In [0]:
!cd libgpuarray; python setup.py build;
!cd libgpuarray; python setup.py install

You can test the installation by restarting the notebook (Runtime/Restart runtime) and execute the code in the first cell again. It should say "used the **gpu**".


And now we just wait until PyMC4 has all the funtionality we need... In the mean time: have fun with PyMC3 in Colab!